# Sistema Nacional de Información e Indicadores de Vivienda

ID |Descripción
---|:----------
P0405|Viviendas Verticales
P0406|Viviendas urbanas en PCU U1 y U2
P0411|Subsidios CONAVI

In [1]:
descripciones = {
    'P0405': 'Viviendas Verticales',
    'P0406': 'Viviendas urbanas en PCU U1 y U2',
    'P0411': 'Subsidios CONAVI'
}

In [17]:
# Librerias utilizadas
import pandas as pd
import sys
import urllib
import os
import csv
import zeep
import requests
from lxml import etree
import xmltodict
import ast
import collections

In [3]:
# Configuracion del sistema
print('Python {} on {}'.format(sys.version, sys.platform))
print('Pandas version: {}'.format(pd.__version__))
import platform; print('Running on {} {}'.format(platform.system(), platform.release()))

Python 3.6.1 |Anaconda 4.4.0 (64-bit)| (default, May 11 2017, 13:25:24) [MSC v.1900 64 bit (AMD64)] on win32
Pandas version: 0.20.1
Running on Windows 8.1


## 2. Descarga de datos
Los datos se descargan por medio de una conexión a un servicio SOAP proporcionado por el SNIIV. Para acceder a los datos proporcionados por cada uno de los servicios del SNIIV, tiene que hacerse un POST request, especificando en el encabezado el servicio al que se busca acceder y en el cuerpo de la petición, un XML con parametros para que el servidor de SNIIV pueda regresar una respuesta 

In [4]:
# Esta celda contiene textos estándar para los encabezados y el cuerpo de la operación que se solicita al servidor.
# los textos entre corchetes {} sirven para especificar la operacion a la que se busca tener acceso
scheme = r'http://www.conavi.gob.mx:8080/WS_App_SNIIV.asmx?WSDL'       #El scheme siempre es el mismo
SOAPAction = ('http://www.conavi.gob.mx:8080/WS_App_SNIIV/{}')
xmlbody = (
'<?xml version="1.0" encoding="utf-8"?>'
'<soap:Envelope xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xmlns:xsd="http://www.w3.org/2001/XMLSchema" xmlns:soap="http://schemas.xmlsoap.org/soap/envelope/">'
  '<soap:Body>'
    '<{} xmlns="http://www.conavi.gob.mx:8080/WS_App_SNIIV">'
      '<dat></dat>'
    '</{}>'
  '</soap:Body>'
'</soap:Envelope>'
)


In [5]:
# Conexion y descarga de datos
operacion = 'Subsidios'
heads = {'Content-Type': 'text/xml; charset=utf-8', 
           'SOAPAction': SOAPAction.format(operacion)}
body = xmlbody.format(operacion, operacion)
r = requests.post(scheme, data=body, headers=heads)
print(r.content[0:300])

b'<?xml version="1.0" encoding="utf-8"?><soap:Envelope xmlns:soap="http://schemas.xmlsoap.org/soap/envelope/" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xmlns:xsd="http://www.w3.org/2001/XMLSchema"><soap:Body><SubsidiosResponse xmlns="http://www.conavi.gob.mx:8080/WS_App_SNIIV"><SubsidiosRe'


In [6]:
list(r.headers.keys())

['Cache-Control',
 'Content-Type',
 'Server',
 'X-AspNet-Version',
 'X-Powered-By',
 'Date',
 'Content-Length']

In [7]:
r = xmltodict.parse(r.content)
r.keys()

odict_keys(['soap:Envelope'])

In [8]:
# Después de parseada la respuesta a un diccionario de Python, los datos se encuentran varios niveles por debajo, por lo que es 
# necesario explorar estos niveles hasta llegar a los datos útiles
rodict = r['soap:Envelope']['soap:Body']['{}Response'.format(operacion)]['{}Result'.format(operacion)]['app_sniiv_rep_subs']
rodict[0]

OrderedDict([('cve_ent', '01'),
             ('tipo_ee', 'NO DISPONIBLE'),
             ('modalidad', 'Autoproducción'),
             ('acciones', '11'),
             ('monto', '781626.56')])

In [9]:
# Con los datos parseados en forma de OrderedDict ya es posible hacer un DataFrame de la siguiente manera.
pd.DataFrame(rodict, columns=rodict[0].keys()).head()

,cve_ent,tipo_ee,modalidad,acciones,monto
0,01,NO DISPONIBLE,Autoproducción,11,781626.56
1,01,INFONAVIT,Nueva,234,13556833.79
2,01,INFONAVIT,Usada,6,318407.02
3,01,RIF - FOVI,Nueva,1,47500
4,02,INFONAVIT,Nueva,280,12989084.57


Lo anterior es un ejemplo de cómo pueden obtenerse datos desde el servicio montado por el SNIIV. Con base en este ejemplo es posible hacer una función que realice las peticiones de manera más compacta en cada caso.

In [30]:
def getsoap(operacion):
    scheme = r'http://www.conavi.gob.mx:8080/WS_App_SNIIV.asmx?WSDL'       # El scheme siempre es el mismo
    SOAPAction = ('http://www.conavi.gob.mx:8080/WS_App_SNIIV/{}')
    xmlbody = (
    '<?xml version="1.0" encoding="utf-8"?>'
    '<soap:Envelope xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xmlns:xsd="http://www.w3.org/2001/XMLSchema" xmlns:soap="http://schemas.xmlsoap.org/soap/envelope/">'
      '<soap:Body>'
        '<{} xmlns="http://www.conavi.gob.mx:8080/WS_App_SNIIV">'
          '<dat></dat>'
        '</{}>'
      '</soap:Body>'
    '</soap:Envelope>'
    )
    heads = {'Content-Type': 'text/xml; charset=utf-8', 
           'SOAPAction': SOAPAction.format(operacion)}
    body = xmlbody.format(operacion, operacion)
    r = requests.post(scheme, data=body, headers=heads)
    if r.status_code != 200:          # 
        print('status: {}\n**Operacion terminada**'.format(r.status_code))
        return None
    else:
        print(operacion)
        print('status: {}\nContent Type: {}'.format(r.status_code, r.headers['Content-Type']))
        print('Date: {}\nContent Lenght: {}'.format(r.headers['Date'], r.headers['Content-Length']))
        r = xmltodict.parse(r.content)
        try: 
            rodict = r[list(test.keys())[0]]['soap:Body']['{}Response'.format(operacion)]['{}Result'.format(operacion)][list(test.keys())[0]]
            df = pd.DataFrame(rodict, columns=rodict[0].keys())
            print('dataframe creado')
            return df
        except:
            print('---------\nNo fue posible crear dataframe. Regresando OrderedDict')
            return r

In [11]:
# Prueba del Script
test = getsoap('get_tot_fech')
test

get_tot_fech
status: 200
Content Type: text/xml; charset=utf-8
Date: Fri, 20 Apr 2018 18:46:02 GMT
Content Lenght: 550
---------
No fue posible crear dataframe. Regresando OrderedDict


OrderedDict([('soap:Envelope',
              OrderedDict([('@xmlns:soap',
                            'http://schemas.xmlsoap.org/soap/envelope/'),
                           ('@xmlns:xsi',
                            'http://www.w3.org/2001/XMLSchema-instance'),
                           ('@xmlns:xsd', 'http://www.w3.org/2001/XMLSchema'),
                           ('soap:Body',
                            OrderedDict([('get_tot_fechResponse',
                                          OrderedDict([('@xmlns',
                                                        'http://www.conavi.gob.mx:8080/WS_App_SNIIV'),
                                                       ('get_tot_fechResult',
                                                        OrderedDict([('app_sniiv_tot_date',
                                                                      OrderedDict([('fecha_finan',
                                                                                    'Datos al 28 de Febrero 2018

Con el script creado es posible iterar por todas las operaciones que tiene el servidor de SNIIV para explorar los datos disponibles y extraer datos para parámetros.

In [12]:
operaciones = {
    'viv_vig_x_avnc': 'Descripción: Obtiene la Vivienda Vigente por Avance de Obra a la última fecha de actualización, como parámetro pase una cadena vacía',
    'get_tot_fech': 'Descripción: Obtiene las últimas fecha de actualización de la información referente a financiamientos, subsidios y vivienda vigente, como parámetro pase una cadena vacía',
    'Financiamientos': 'Descripción: Obtiene datos de los Financiamientos por Organismo, Destino y Agrupación a la última fecha de actualización, como parámetro pase una cadena vacía',
    'viv_vig_x_pcu': 'Descripción: Obtiene la Vivienda Vigente por PCU a la última fecha de actualización, como parámetro pase una cadena vacía',
    'Subsidios': 'Descripción: Obtiene datos de los Subsidios CONAVI por Tipo de Entidad Ejecutora y Modalidad a la última fecha de actualización, como parámetro pase una cadena vacía',
    'viv_vig_x_tipo': 'Descripción: Obtiene la Vivienda Vigente por Tipo a la última fecha de actualización, como parámetro pase una cadena vacía',
    'get_tot_ini': 'Descripción: Obtiene el total de financiamientos, subsidios y vivienda vigente a la última fecha de actualización (acciones y monto), como parámetro pase una cadena vacía',
    'viv_vig_x_valor': 'Descripción: Obtiene la Vivienda Vigente por Valor a la última fecha de actualización, como parámetro pase una cadena vacía',
    'financiamientos_gpo_org': 'Descripción: Obtiene datos de los Financiamientos por Grupo y Organismo a nivel Nacional y Estatal a la última fecha de actualización, como parámetro pase una cadena vacía',
    'get_avnc_vv_mun': 'Descripción: Obtiene la Oferta de Vivienda Vigente a nivel Municipio (Top 3) por Avance de Construcción',
    'get_cont_vv_mun': 'Descripción: Obtiene la Oferta de Vivienda Vigente a nivel Municipio (Top 3) por PCU',
    'get_fechas_act': 'Descripción: Obtiene últimas fecha de actualización de la información referente a financiamientos, subsidios y vivienda vigente (dd/mm/aaaa), como parámetro pase una cadena vacía',
    'get_finan_evol': 'Descripción: Obtiene la evolución de Acciones y Monto de Financiamientos en los últimos 3 años (mes a mes)',
    'get_finan_evol_acum': 'Descripción: Obtiene la evolución de Acciones y Monto de Financiamientos en los últimos 3 años (acumulado)',
    'get_finan_rg_mun': 'Descripción: Obtiene el Reporte General de Financiamientos a nivel Municipio (Top 3)',
    'get_finan_x_rgoing': 'Descripción: Obtiene Acciones y Monto de Financiamientos por Rango de Ingreso VSMM a Nivel Estatal',
    'get_finan_x_rgoing_mun': 'Descripción: Obtiene Acciones y Monto de Financiamientos por Rango de Ingreso VSMM a nivel Municipio (Top 3)',
    'get_finan_x_valviv': 'Descripción: Obtiene Acciones y Monto de Financiamientos por Valor de la Vivienda a Nivel Estatal',
    'get_finan_x_valviv_mun': 'Descripción: Obtiene Acciones y Monto de Financiamientos por Valor de la Vivienda a nivel Municipio (Top 3)',
    'get_regviv_evol': 'Descripción: Obtiene la evolución de Registro de Vivienda en los últimos 3 años (mes a mes)',
    'get_regviv_evol_acum': 'Descripción: Obtiene la evolución de Registro de Vivienda en los últimos 3 años (acumulado)',
    'get_subs_evol': 'Descripción: Obtiene la evolución de Acciones y Monto de Subsidios en los últimos 3 años (mes a mes)',
    'get_subs_evol_acum': 'Descripción: Obtiene la evolución de Acciones y Monto de Subsidios en los últimos 3 años (acumulado)',
    'get_subs_rg_mun': 'Descripción: Obtiene el Reporte General de Subsidios a nivel Municipio (Top 3)',
    'get_subs_x_rgoing': 'Descripción: Obtiene Acciones y Monto de Subsidios CONAVI por Rango de Ingreso VSMM a Nivel Estatal',
    'get_subs_x_rgoing_mun': 'Descripción: Obtiene Acciones y Monto de Subsidios CONAVI por Rango de Ingreso VSMM a nivel Municipio (Top 3)',
    'get_subs_x_valviv': 'Descripción: Obtiene Acciones y Monto de Subsidios CONAVI por Valor de la Vivienda a Nivel Estatal',
    'get_subs_x_valviv_mun': 'Descripción: Obtiene Acciones y Monto de Subsidios CONAVI por Valor de la Vivienda a nivel Municipio (Top 3)',
    'get_tipohv_vv_mun': 'Descripción: Obtiene la Oferta de Vivienda Vigente a nivel Municipio (Top 3) por Tipo de Vivienda Horizontal - Vertical',
    'get_tipol_vv_mun': 'Descripción: Obtiene la Oferta de Vivienda Vigente a nivel Municipio (Top 3) por Tipología de Vivienda'
}

In [31]:
# Iterar por todas las operaciones
datos = {}
for operacion, descripcion in operaciones.items():
    print('{}\n{}:\n{}'.format('||'*30, operacion, descripcion))
    datos[operacion] = getsoap(operacion)

||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||
viv_vig_x_avnc:
Descripción: Obtiene la Vivienda Vigente por Avance de Obra a la última fecha de actualización, como parámetro pase una cadena vacía
viv_vig_x_avnc
status: 200
Content Type: text/xml; charset=utf-8
Date: Fri, 20 Apr 2018 19:01:17 GMT
Content Lenght: 8422
---------
No fue posible crear dataframe. Regresando OrderedDict
||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||
get_tot_fech:
Descripción: Obtiene las últimas fecha de actualización de la información referente a financiamientos, subsidios y vivienda vigente, como parámetro pase una cadena vacía
get_tot_fech
status: 200
Content Type: text/xml; charset=utf-8
Date: Fri, 20 Apr 2018 19:01:17 GMT
Content Lenght: 550
---------
No fue posible crear dataframe. Regresando OrderedDict
||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||
Financiamientos:
Descripción: Obtiene datos de los Financiamientos por Organismo, Destino y Agrupación a la últim

get_subs_evol
status: 200
Content Type: text/xml; charset=utf-8
Date: Fri, 20 Apr 2018 19:01:19 GMT
Content Lenght: 48017
---------
No fue posible crear dataframe. Regresando OrderedDict
||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||
get_subs_evol_acum:
Descripción: Obtiene la evolución de Acciones y Monto de Subsidios en los últimos 3 años (acumulado)
get_subs_evol_acum
status: 200
Content Type: text/xml; charset=utf-8
Date: Fri, 20 Apr 2018 19:01:19 GMT
Content Lenght: 49272
---------
No fue posible crear dataframe. Regresando OrderedDict
||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||
get_subs_rg_mun:
Descripción: Obtiene el Reporte General de Subsidios a nivel Municipio (Top 3)
get_subs_rg_mun
status: 200
Content Type: text/xml; charset=utf-8
Date: Fri, 20 Apr 2018 19:01:19 GMT
Content Lenght: 41567
---------
No fue posible crear dataframe. Regresando OrderedDict
||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||
get_subs_x_rgoing:
Descripción: 

**Se van a revisar los siguientes:**

get_subs_x_valviv_mun - Subsidios CONAVI

get_subs_x_rgoing_mun - Subsidios CONAVI

get_regviv_evol_acum - Vivienda en PCU

get_cont_vv_mun - Vivienda en PCU

get_tipohv_vv_mun - Vivienda Vigente por tipo

ID |Descripción
---|:----------
P0405|Viviendas Verticales
P0406|Viviendas urbanas en PCU U1 y U2
P0411|Subsidios CONAVI

# Subsidios CONAVI
Los datos para este indicador pueden estar en:

get_subs_x_valviv_mun

get_subs_x_rgoing_mun

In [14]:
# Agrupadores estándar para todos los xml
senv = 'soap:Envelope'
sbo = 'soap:Body'

In [15]:
revdata = datos['get_subs_x_valviv_mun'][senv][sbo]['get_subs_x_valviv_munResponse']['get_subs_x_valviv_munResult']
revdata 


'{"01":[{"AGUASCALIENTES":[{"valor_vivienda":"Económica","acciones":"0","monto":"0"},{"valor_vivienda":"Popular","acciones":"208","monto":"12478137.96"},{"valor_vivienda":"Tradicional","acciones":"0","monto":"0"},{"valor_vivienda":"Media","acciones":"0","monto":"0"},{"valor_vivienda":"Residencial","acciones":"0","monto":"0"},{"valor_vivienda":"Residencial Plus","acciones":"0","monto":"0"},{"valor_vivienda":"No disponible","acciones":"0","monto":"0"},{"valor_vivienda":"Total","acciones":"208","monto":"12478137.96"}],"SAN FRANCISCO D":[{"valor_vivienda":"Económica","acciones":"34","monto":"1690150.93"},{"valor_vivienda":"Popular","acciones":"0","monto":"0"},{"valor_vivienda":"Tradicional","acciones":"0","monto":"0"},{"valor_vivienda":"Media","acciones":"0","monto":"0"},{"valor_vivienda":"Residencial","acciones":"0","monto":"0"},{"valor_vivienda":"Residencial Plus","acciones":"0","monto":"0"},{"valor_vivienda":"No disponible","acciones":"0","monto":"0"},{"valor_vivienda":"Total","acciones

In [16]:
# Los datos que regresó el servidor para esta operación parecen ser un JSON, pero vienen en forma de str
type(revdata)

str

In [22]:
revdic = ast.literal_eval(revdata)
revdic = collections.OrderedDict(revdic)
revdic

OrderedDict([('01',
              [{'AGUASCALIENTES': [{'acciones': '0',
                  'monto': '0',
                  'valor_vivienda': 'Económica'},
                 {'acciones': '208',
                  'monto': '12478137.96',
                  'valor_vivienda': 'Popular'},
                 {'acciones': '0',
                  'monto': '0',
                  'valor_vivienda': 'Tradicional'},
                 {'acciones': '0', 'monto': '0', 'valor_vivienda': 'Media'},
                 {'acciones': '0',
                  'monto': '0',
                  'valor_vivienda': 'Residencial'},
                 {'acciones': '0',
                  'monto': '0',
                  'valor_vivienda': 'Residencial Plus'},
                 {'acciones': '0',
                  'monto': '0',
                  'valor_vivienda': 'No disponible'},
                 {'acciones': '208',
                  'monto': '12478137.96',
                  'valor_vivienda': 'Total'}],
                'JESUS MARIA': [

In [24]:
revdic.keys()

odict_keys(['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32'])

In [26]:
pd.DataFrame(revdic2['01'], columns=revdic2['01'][list(revdic2['01'].keys())])

In [61]:
revdic2['01']['AGUASCALIENTES']

[{'acciones': '0', 'monto': '0', 'valor_vivienda': 'Económica'},
 {'acciones': '208', 'monto': '12478137.96', 'valor_vivienda': 'Popular'},
 {'acciones': '0', 'monto': '0', 'valor_vivienda': 'Tradicional'},
 {'acciones': '0', 'monto': '0', 'valor_vivienda': 'Media'},
 {'acciones': '0', 'monto': '0', 'valor_vivienda': 'Residencial'},
 {'acciones': '0', 'monto': '0', 'valor_vivienda': 'Residencial Plus'},
 {'acciones': '0', 'monto': '0', 'valor_vivienda': 'No disponible'},
 {'acciones': '208', 'monto': '12478137.96', 'valor_vivienda': 'Total'}]

In [59]:
revdic2 = {}
for estado in revdic.keys():
    tempdict = {}
    revdic2[estado] = collections.OrderedDict(revdic[estado][0])
revdic2['01']

OrderedDict([('AGUASCALIENTES',
              [{'acciones': '0', 'monto': '0', 'valor_vivienda': 'Económica'},
               {'acciones': '208',
                'monto': '12478137.96',
                'valor_vivienda': 'Popular'},
               {'acciones': '0',
                'monto': '0',
                'valor_vivienda': 'Tradicional'},
               {'acciones': '0', 'monto': '0', 'valor_vivienda': 'Media'},
               {'acciones': '0',
                'monto': '0',
                'valor_vivienda': 'Residencial'},
               {'acciones': '0',
                'monto': '0',
                'valor_vivienda': 'Residencial Plus'},
               {'acciones': '0',
                'monto': '0',
                'valor_vivienda': 'No disponible'},
               {'acciones': '208',
                'monto': '12478137.96',
                'valor_vivienda': 'Total'}]),
             ('SAN FRANCISCO D',
              [{'acciones': '34',
                'monto': '1690150.93',
       